How to set up selenium and client?

In [ ]:
var importer = require('../Core');

var client, results, getCredentials, loginGoogle, getAllXPath;
var getClient = () => {
    return importer.interpretAll([
        'webdriver client',
        'decrypt password',
        'log in Google',
        'get all elements'
    ])
    .then(r => {
        results = r;
        return r[0].runInNewContext();
    })
    .then(r => {
        client = r();
        getCredentials = results[1].runInNewContext({client});
        loginGoogle = results[2].runInNewContext({client, getCredentials});
        getAllXPath = results[3].runInNewContext({client});
        return client;
    });
};
module.exports = getClient;
getClient;



In [ ]:
$$.async();
getClient()
    .then(c => $$.sendResult(c))
    .catch(e => $$.sendError(e));


Scrape google timeline?



In [ ]:
var getGoogleTimeline = () => {
    console.log('Google: Logging timeline history');
    return client
    .url('https://www.google.com/maps/timeline')
    .loginGoogle()
    .waitUntil(() => client.getUrl()
               .then(url => url.indexOf('timeline') > -1, 20000, ''))
    .catch(function (e) {
        console.log(e);
    })
    .pause(3000)
    .click('button[jsaction="header.select-today"]')
    .pause(2000);
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('getGoogleTimeline', getGoogleTimeline);
}
module.exports = getGoogleTimeline;
getGoogleTimeline;


Read Google timeline?



In [ ]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = '/Users/briancullinan/Timeline';

var readTimelinePage;
var readAllPages = () => {
    // TODO: add start/stop conditions
    const oneDay = 24*60*60*1000; // hours*minutes*seconds*milliseconds
    var firstDate = new Date(2016,8,21);
    var secondDate = new Date();
    var days = Math.round(
        Math.abs((firstDate.getTime() - secondDate.getTime())/(oneDay)));
    var promises = [];
    for (var c = 0; c < days; c++) {
        promises[promises.length] = (resolve) => {
            var result;
            return client
            .then(() => typeof readTimelinePage !== 'undefined'
                ? []
                : importer.interpretAll(['single timeline page'])
// TODO: fix this with intend() which runs the interpretation in one call
            .then(r => (readTimelinePage = r[0].runInNewContext({client}))))
            
            .then(() => readTimelinePage())
            .then(r => result = r)
            .click('.previous-date-range-button')
            .pause(3000)
            .then(() => {
                if(typeof result == 'undefined') {
                    resolve();
                }
                fs.writeFileSync(
                    PROJECT_PATH + '/timeline-' + result[0].timeline + '_.json',
                    JSON.stringify(result, null, 4))
                resolve(result);
            })
        };
    }
    return promises;
}
if (typeof client.getGoogleTimeline == 'undefined') {
    client.addCommand('readAllPages', readAllPages);
}
module.exports = readAllPages;
readAllPages;


Read single google timeline page?



In [ ]:
var chrono = require('chrono-node');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var readTimelinePage = () => {
    return client.getAllXPath({
        day: ['//*[contains(@class,"timeline-subtitle")]//text()|//*[contains(@class,"timeline-title")]//text()'],
        items: [
            '//*[contains(@class,"timeline-item")]/parent::*/*[@jsinstance]',
            {
                duration: './/*[contains(@class, "duration-text")]//text()',
                data: './/*[contains(@class, "timeline-item")]/@data-segment-key',
                title: './/*[contains(@class, "timeline-item-title-content")][.//i]/*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-title-content")][not(.//i)]//text()',
                location: './/*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][.//a]/a//*[not(self::i)]//text()|.//*[contains(@class, "timeline-item-text")][not(contains(@class,"add-child"))][not(.//a)]/text()'
            }
        ]
    })
    .then(r => {
        if(r.day.length == 0) {
            return;
        }
        var currDate = chrono.parseDate(r.day[0] + '')
            || chrono.parseDate(r.day[1] + '');
        var newKey = currDate.getDate() 
            + months[currDate.getMonth()] 
            + (currDate.getFullYear() + '').substr(2, 2);
        return r.items.map(i => {
            var timelineData = (i.data + '').split(':');
            var start, end, length;
            if(timelineData.length == 3) {
                start = new Date(parseFloat(timelineData[1]));
                end = new Date(parseFloat(timelineData[2]));
                length = end.getTime() - start.getTime();
            } else {
                start = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[0]);
                end = chrono.parseDate(r.day + ' ' + i.duration.join('').trim().split('-')[1]);
                length = 0;
            }
            var traveling = (/(Driving|Walking|Traveling|Flying|Moving).*\s+-\s+(.*),/ig).exec(i.title + ', ' + i.location)
            return ({
                traveling: traveling ? traveling[0] : false,
                type: 'timeline',
                timeline: newKey,
                name: i.title,
                location: i.location,
                time: start,
                length: isNaN(length) ? 0 : length
            })
        })
    })
    .catch(e => console.log(e))
};
if (typeof client.readTimelinePage == 'undefined') {
    client.addCommand('readTimelinePage', readTimelinePage);
}
module.exports = readTimelinePage;
readTimelinePage;


Load location export data?



In [ ]:
var importer = require('../Core');

var months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'];
var loadLocations = (file) => {
    var locationCache = {};
    return importer.streamJson(file, [true, {emitPath: true}], (match) => {
        if(match.path[0] === 'locations') {
            var currDate = new Date(parseInt(match.value.timestampMs));
            var newKey = currDate.getDate() + months[currDate.getMonth()]
                + (currDate.getFullYear() + '').substr(2, 2);
            var newRow = Object.assign({
                    time: currDate,
                    type: 'location',
                    location: newKey,
            }, match.value);
            var cache = locationCache[newRow.location];
            if(typeof cache === 'undefined') {
                cache = locationCache[newRow.location] = [];
            }
            cache[cache.length] = newRow;
        }
    })
    .then(() => locationCache)
};
module.exports = loadLocations;
loadLocations;



Find the average latitute and longitude at each destination?



In [ ]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = PROFILE_PATH + '/Timeline';

function toRadians (angle) {
  return angle * (Math.PI / 180);
}

var straightDistance = (lat1, lon1, lat2, lon2) => {
    var R = 6371e3; // metres
    var φ1 = toRadians(lat1);
    var φ2 = toRadians(lat2);
    var Δφ = toRadians(lat2-lat1);
    var Δλ = toRadians(lon2-lon1);

    var a = Math.sin(Δφ/2) * Math.sin(Δφ/2) +
            Math.cos(φ1) * Math.cos(φ2) *
            Math.sin(Δλ/2) * Math.sin(Δλ/2);
    var c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));

    var d = R * c;
    return d;
}

var averageDestinations = (geoLocations, timelineLocations) => {
    var destinations = [];
    for(const d of timelineLocations) {
        if((d.name + '') == '' || (d.name + '').indexOf('Driving') > -1) {
            continue;
        }
        const nearest = _.sortBy(
            geoLocations,
            l => Math.abs(new Date(l.time).getTime() - new Date(d.time).getTime()))
            .slice(0, 3);
        // make sure it isn't off by much
        const averageLat = nearest.map(n => n.latitudeE7)
            .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
        const averageLon = nearest.map(n => n.longitudeE7)
            .reduce((a, b) => a + b, 0) / nearest.length / 10000000;
        if(nearest.filter(nearby => straightDistance(
                nearby.latitudeE7 / 10000000,
                nearby.longitudeE7 / 10000000,
                averageLat,
                averageLon
            ) > 500).length > 0) {
            destinations.push(Object.assign(d, {
                averageLat: averageLat,
                averageLon: averageLon,
                locations: nearest.map(nearby => Object.assign(nearby, {
                    averageLat: nearby.latitudeE7 - averageLat,
                    averageLon: nearby.longitudeE7 - averageLon,
                    averageDist: straightDistance(
                        nearby.latitudeE7 / 10000000,
                        nearby.longitudeE7 / 10000000,
                        averageLat,
                        averageLon
                    )
                }))
            }));
        }
        destinations.push(Object.assign(d, {
            averageLat: averageLat,
            averageLon: averageLon
        }));
    }
    return destinations;
}
module.exports = averageDestinations;
averageDestinations;



Reconcile timeline data with calendar?



In [1]:
var importer = require('../Core');
var fs = require('fs');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = '/Users/briancullinan/Timeline';


var getEvents = (time) => {
    const start = new Date(time);
    start.setHours(0, 0, 0);
    start.setHours(start.getHours() + 7);
    const end = new Date(time);
    end.setHours(23, 59, 0);
    end.setHours(end.getHours() + 7);
    return listEvents({
        auth: options.auth,
        timeMin: start.toString(),
        timeMax: end.toString(),
    })
    .catch(e => console.log(e));
}

var filterEvents = (events, locations) => {
    const unmatched = [], matches = [];
    for(const l of locations) {
        const matching = events.filter(e => {
            return ((l.name + '').indexOf('Gainey') > -1
                && e.event.summary == 'Drive to work')
            || ((l.name + '').indexOf('Swiftpage') > -1
                && e.event.summary == 'Drive to work')
            || ((l.name + '').indexOf('6934') > -1
                && e.event.summary == 'Drive home')
            || ((l.name + '').indexOf('6934') > -1
                && e.event.summary == 'Work from home')
            || (typeof e.event.location !== 'undefined'
                && ((e.event.location + '').indexOf(l.location + '') > -1
                || (e.event.location + '').indexOf(l.name + '') > -1))
        });
        if(matching.length == 0) {
            unmatched.push(l);
        } else {
            const result = {};
            Object.assign(result, l);
            Object.assign(result, matching[0]);
            matches.push(result);
        }
    }
    console.log(events.map(e => e.event.summary + ' - ' + e.event.location))
    console.log('Unmatched ' + unmatched.length
                + ' -  out of ' + matches.length
                + ' - ' + unmatched.map(u => u.name + '').join(', '));
    return unmatched;
}

var listEvents, options = {}, nearbyCache;
try {
    nearbyCache = JSON.parse(fs.readFileSync(
         PROJECT_PATH + '/geolocations.json').toString());
} catch (e) {
    nearbyCache = {};
}
var reconcileTimeline = (destinations, date) => {
    var placesNearby, getOauthClient, events, locations = [];
    return importer.interpretAll([
        'import google calendar api',
        'places nearby google maps',
        'list events',
    ])
    .then(r => {
        getOauthClient = r[0].runInNewContext();
        placesNearby = r[1].runInNewContext();
        listEvents = r[2].runInNewContext();
        return getOauthClient(options);
    })
    .then(() => getEvents(date))
    .then(r => {
        events = r;
        return importer.runAllPromises(destinations
                                       .filter(d => !d.traveling)
                                       .map(d => resolve => {
            if(typeof nearbyCache[d.name + ', ' + d.location] !== 'undefined') {
                return resolve(nearbyCache[d.name + ', ' + d.location]);
            }
            console.log(d.name + ', ' + d.location);
            return placesNearby(d.name + ', ' + d.location,
                            {lat: d.averageLat, lng: d.averageLon})
            .then(result => {
                if(result.length === 0) {
                    console.warn('No match for ' + JSON.stringify(d))
                    resolve();
                } else {
                    nearbyCache[d.name + ', ' + d.location] = result[0];
                    resolve(Object.assign(d, result[0]))
                }
            })
            .catch(e => resolve(e))
        }))
    })
    .then(r => {
        fs.writeFileSync(
            PROJECT_PATH + '/geolocations.json',
            JSON.stringify(nearbyCache, null, 4));
        locations = r.filter(l => typeof l !== 'undefined');
        return filterEvents(events, locations);
    })
    .catch(e => console.log(e))
}
module.exports = reconcileTimeline;
reconcileTimeline;



[Function: reconcileTimeline]

In [6]:
var importer = require('../Core');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var PROJECT_PATH = '/Users/briancullinan/Timeline';

var loadLocations, averageDestinations, reconcileTimeline, runSeleniumCell, geoLocations, cancelled = false;

$$.async();
importer.interpretAll([
    'load locations export data',
    'average latitude longitude',
    'reconcile timeline calendar',
    'selenium cell'
])
.then(r => {
    loadLocations = r[0].runInNewContext();
    averageDestinations = r[1].runInNewContext();
    reconcileTimeline = r[2].runInNewContext();
    runSeleniumCell = r[3].runInNewContext();
})
.then(() => {
    return loadLocations('/Users/briancullinan/Downloads/Takeout 7/Location History/Location History.json');
})
.then(geo => {
    geoLocations = geo;
    return runSeleniumCell([
        'log in google',
        'read google timeline',
        'scrape google timeline',
    ]);
})
.then(r => {
    readAllPages = r[1];
    getGoogleTimeline = r[2];
})
.then(() => getGoogleTimeline())
.then(() => importer.runAllPromises(readAllPages().map(promise => resolve => {
    if(cancelled) return resolve();
    return new Promise(promise)
    .catch(e => console.log(e))
    .then(timelineLocations => {
        if(timelineLocations.length == 0) return;
        var date = timelineLocations[0].timeline;
        if(typeof geoLocations[date] == 'undefined') return;
        /*
        if(!fs.existsSync(PROJECT_PATH + '/timeline-' + date + '_.json')) {
            return;
        }
        var timelineData = JSON.parse(
            fs.readFileSync(PROJECT_PATH + '/timeline-' + date + '_.json')
            .toString());
        */
        console.log(date);
        var d = averageDestinations(geoLocations[date], timelineLocations);
        if(d.length == 0) {
            return;
        }
        return reconcileTimeline(d, date);
    })
    .then(r => {
        if(typeof r !== 'undefined' && r.length > 0) cancelled = true;
        resolve(r);
    })
    .catch(e => console.log(e))
})))
.then(r => $$.sendResult(r.filter(o => typeof o !== 'undefined')))
.catch(e => $$.sendError(e))



Initializing webdriver on localhost
Google: Logging timeline history
29Sep17
[ 'Drive to work - undefined',
  'Work on eloqua service - undefined',
  'Work on renewals service, unit testing, compare status fields with Zuora data dump - undefined',
  'Lunch with Iga - Pei Wei, 8787 N Scottsdale Rd. 214/216, Scottsdale, AZ 85253, USA',
  'Eat more vegetables - undefined',
  'Duplicate git repos independent of organization using patch files - undefined',
  'Drive home - undefined',
  'Study lynda - undefined',
  'Pick up Aneesh - 6440 E Camino De Los Ranchos, Scottsdale, AZ 85254, USA',
  'Data Science Dinner III - Culinary Dropout, 5632 N 7th St, Phoenix, AZ 85014, USA',
  'Hung out with Aneesh, Iga, David, Abe with glasses - undefined',
  'Stop Spending Money - undefined',
  'Go offline - undefined' ]
Unmatched 0 -  out of 11 - 
28Sep17
[ 'Drive to work - undefined',
  'Work on portal, cart paths, i18n - undefined',
  'Eat more vegetables - undefined',
  'Work on portal, responsiveness 

[ { geometry: { location: [Object], viewport: [Object] },
    icon: 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
    id: '975ff03b400ca247293dbaf9bd3b4dfc2193cdda',
    name: '6934 E Sandra Terrace',
    place_id: 'ChIJzfjYxhh0K4cR8Opz6XxNFnQ',
    reference: 'CmRbAAAAsrEDGcDegqau80e1PQ9d6Qk1-P28BiiAFrfzsoBY2bxTEPDC5KKNJB1WCcG4OcCk10uoIFoBAwTa0JYr-oSHXhwG1c3kSizI_uUgutz1VgKMp8SKrCOBunGIfbpO6CViEhCyCj-g0rBcTYAYxugjFN1xGhQ2m4lNPJT6u95K51hQ51AH9WsXPA',
    scope: 'GOOGLE',
    types: [ 'premise' ],
    vicinity: 'Scottsdale' },
  { geometry: { location: [Object], viewport: [Object] },
    icon: 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png',
    id: '975ff03b400ca247293dbaf9bd3b4dfc2193cdda',
    name: '6934 E Sandra Terrace',
    place_id: 'ChIJzfjYxhh0K4cR8Opz6XxNFnQ',
    reference: 'CmRbAAAAsrEDGcDegqau80e1PQ9d6Qk1-P28BiiAFrfzsoBY2bxTEPDC5KKNJB1WCcG4OcCk10uoIFoBAwTa0JYr-oSHXhwG1c3kSizI_uUgutz1VgKMp8SKrCOBunGIfbpO6CViEhCyCj-g0rBcTYAYxugjFN1xGh